# Рекомендательная система на библиотеке surprise (Практика 2)

Сайт библиотеки: http://surpriselib.com/

Установки библиотеки:
```
$ pip install scikit-surprise
```
или
```
$ conda install -c conda-forge scikit-surprise```

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("recdemo.csv", sep=";") #небольшой dataset

In [3]:
df

,id,A,B,C,D,E,F
0,1,3.0,4.0,2.0,4.0,1.0,NaN
1,2,3.0,4.0,2.0,4.0,NaN,2.0
2,3,NaN,2.0,5.0,5.0,NaN,5.0
3,4,NaN,NaN,NaN,NaN,4.0,NaN
4,5,3.0,NaN,2.0,4.0,4.0,NaN
5,6,NaN,5.0,5.0,5.0,NaN,5.0
6,7,1.0,NaN,NaN,2.0,NaN,3.0
7,8,NaN,NaN,NaN,NaN,4.0,4.0


In [4]:
df_unpivot=pd.melt(df, id_vars=['id'])
df_unpivot.head()

,id,variable,value
0,1,A,3.0
1,2,A,3.0
2,3,A,NaN
3,4,A,NaN
4,5,A,3.0


In [5]:
df_unpivot.dropna(inplace=True)
df_unpivot.columns=['userID', 'itemID', 'rating']

In [6]:
from surprise import Dataset
from surprise import Reader


In [7]:
reader = Reader(rating_scale=(1, 5)) # Зададим разброс оценок
data = Dataset.load_from_df(df_unpivot[['userID', 'itemID', 'rating']], reader) #создадим объект, с которым умеет работать библиотека

In [8]:
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

$$ x_{ij}=μ+b_i+b_j+q^T_jp_i $$

https://surprise.readthedocs.io/en/stable/matrix_factorization.html

In [9]:
from surprise import SVD  

In [10]:
algo = SVD(n_factors=2,random_state=999)
predictions = algo.fit(trainset).test(testset)

In [11]:
df_unpivot1 = df_unpivot.copy()
for i in predictions:
    df_unpivot1 = df_unpivot1.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)

In [12]:
df_unpivot1.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,3.312456
2,3.000000,4.000000,2.000000,4.000000,3.210599,2.000000
3,3.444350,2.000000,5.000000,5.000000,3.684146,5.000000
4,3.242607,3.599635,3.415230,3.784069,4.000000,3.647434
5,3.000000,3.490772,2.000000,4.000000,4.000000,3.533499
6,3.676756,5.000000,5.000000,5.000000,3.884204,5.000000
7,1.000000,3.194893,2.985395,2.000000,3.010203,3.000000
8,3.301363,3.630884,3.442314,3.798689,4.000000,4.000000


In [13]:
algo.pu # пользователи

array([[-0.02356038,  0.086795  ],
       [ 0.03311668, -0.06586878],
       [-0.01663012, -0.05196951],
       [ 0.17326108, -0.22112997],
       [ 0.11922751,  0.0574848 ],
       [-0.03847042, -0.02430361],
       [-0.10099924, -0.04709509],
       [-0.00974106,  0.08383197]])

In [14]:
algo.qi # фильмы

array([[-0.0499725 ,  0.20377982],
       [ 0.07260589, -0.09175883],
       [-0.0048234 , -0.06959711],
       [-0.08501758, -0.10890099],
       [ 0.16616968,  0.1825492 ],
       [ 0.00912318, -0.15003955]])

In [15]:
algo.qi.T # фильмы

array([[-0.0499725 ,  0.07260589, -0.0048234 , -0.08501758,  0.16616968,
         0.00912318],
       [ 0.20377982, -0.09175883, -0.06959711, -0.10890099,  0.1825492 ,
        -0.15003955]])

In [16]:
mu=df_unpivot.rating.mean()

In [17]:
m= algo.pu @ algo.qi.T 

In [18]:
m.shape

(8, 6)

In [19]:
i = 0
j = 5

In [20]:
mu+m[i][j]+algo.bu[i]+algo.bi[j]

3.31245613193024

## Сравнение параметров

In [21]:
from surprise import KNNBaseline
from surprise.model_selection import cross_validate

В библиотеке есть встроенные Dataset. Можно воспользоваться ими.

Возьмем Dataset movielens (Подробнее о нём https://grouplens.org/datasets/movielens/ )

In [22]:
data = Dataset.load_builtin('ml-100k')##загружем данные о фильмах

In [23]:
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user', 'item', 'rating', 'timestamp']
df.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [24]:
for i in [1,3,5,7,20]:
    algo = KNNBaseline(k=i, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(i)+'NN:',np.mean(cv['test_rmse']))

1NN: 1.2433478973705074
3NN: 1.0418796622725521
5NN: 0.9955741795651495
7NN: 0.9722903280452512
20NN: 0.9407295749363717


In [25]:
for i in [1,2,3,5,7,10,50,70]:
    algo = SVD(n_factors=i,random_state=999, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(i)+'-factors:',np.mean(cv['test_rmse']))

1-factors: 0.9439941673014708
2-factors: 0.9439916818856456
3-factors: 0.9434507437887111
5-factors: 0.9429496588678988
7-factors: 0.9427333823765643
10-factors: 0.9431515582060047
50-factors: 0.9412364669399406
70-factors: 0.9430338895018836


$$ x_{ij}=q^T_jp_i $$

https://surprise.readthedocs.io/en/stable/matrix_factorization.html

In [26]:
for i in [1,2,3,5,7,10,50,70]:
    algo = SVD(n_factors=i,biased=False,random_state=999, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(i)+'-factors:',np.mean(cv['test_rmse']))

1-factors: 0.9683200560414651
2-factors: 0.9666705603001716
3-factors: 0.9596599760160703
5-factors: 0.9592562734193022
7-factors: 0.9577023706170179
10-factors: 0.9544216952059931
50-factors: 0.9544820098490522
70-factors: 0.9603803013939483


## Задание:
```
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.30, random_state=999)
```
1. Для параметров i = [1,5,9,12,17,22] найдите лучший SVD (random_state=999) алгоритм по метрике MAP(n=7).

2. Сделайте прогноз по всем пользователям, который равен среднему из KNN (k=15, c корреляцией Пирсона и item-based подходом) и SVD (n = лучший из пункта 1).
Сравните по метрике MAP(n=7) полученные предсказания с kNN и SVD.